### LLL lattice reduction
A powerful and famous reduction criterion for arbitrary lattice dimensions was introduced by A.K. Lenstra, H.W. Lenstra, and L. Lovász, and the algorithm they proposed is known as the LLL algorithm. It can be interpreted as an extension of Gauss reduction to lattices of rank m > 2.

We provide below a simple LLL implementation in Python

In [1]:
import numpy as np

def LLL(A, delta=0.75):
    # Input: A     : input basis
    #        delta : LLL reduction parameter
    # Output:    B : LLL-reduced basis, B = A*T
    #            T : unimodular transformation matrix
    if delta > 1 or delta < 0.25:
        print("Invalid value of delta, 0.75 used instead")
        delta = 0.75
    # initialization
    B     = np.copy(A)                 # reduced matrix B
    Q,R   = np.linalg.qr(B,'reduced')  # QR decomposition of matrix B
    (n,m) = B.shape                    # matrix dimensions
    T     = np.eye(m)                  # unimodular m x m matrix
    
    # LLL - reduction
    l  = 1
    while l < m:
        # Size reduction of column vector B[:,l]
        for k in range(l-1,-1,-1):
            mu = np.round(R[k,l]/R[k,k])       # abs(R[k,l])>0.5*abs(R[k,k])
            if np.abs(mu)>0:
                B[:,l]     = B[:,l]   - mu * B[:,k]
                R[:k+1,l]  = R[:k+1,l]  - mu * R[:k+1,k]
                T[:,l]     = T[:,l]   - mu * T[:,k]
        # Lovász condition
        lng = np.linalg.norm(R[l-1:l+1,l])
        if delta*np.abs(R[l-1,l-1])**2 > lng**2:
            # swapping of columns l-1 and l in B, T and R
            B[:,[l-1,l]]     = B[:,[l,l-1]]
            T[:,[l-1,l]]     = T[:,[l,l-1]]
            R[:l+1,[l-1,l]]  = R[:l+1,[l,l-1]]
            # reconstruction of upper triangular structure by Givens rotation 
            # mutliplication with matrix Theta achieves R[l,l-1] = 0
            c     = R[l-1,l-1] / lng        # lng = ||R[l-1:l,l-1]|| after swapping
            s     = R[l,l-1]   / lng
            Theta = np.array([[c, s], [-s, c]])
            R[l-1:l+1,l-1:] = np.dot(Theta, R[l-1:l+1,l-1:])
            Q[:,l-1:l+1]      = np.dot(Q[:,l-1:l+1], Theta.T)
            l                 = max(l-1,1)
        else:
            l = l+1

    return B,T

Check our code with a simple example from [here](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm).

In [2]:
# test example from Wikipedia
A = np.array([[1,-1,3],[1,0,5],[1,2,6]])
print("initial lattice basis:")
print(A)
# make LLL reduction
B,T = LLL(A)
print("LLL reduced basis:")
print(B)
print("unimodular transformation matrix:")
print(T)

initial lattice basis:
[[ 1 -1  3]
 [ 1  0  5]
 [ 1  2  6]]
LLL reduced basis:
[[ 0  1 -1]
 [ 1  0  0]
 [ 0  1  2]]
unimodular transformation matrix:
[[-4.  5.  0.]
 [-1.  1.  1.]
 [ 1. -1.  0.]]


Now we calculate a Babai point using Babai's rounding techique.

In [3]:
# input point
t = np.array([4.2,7.1,-2.6])
print("input point t: ")
print(t)
# Babai's rounding technique
l = np.dot(t,np.linalg.inv(B))
v = np.dot(np.round(l),B)
print("Babai point v: ")
print(v)
tv = t-v
print("Euclidean norm of t-v: ")
print("{:.4f}".format(np.linalg.norm(t-v)))
m = np.dot(tv,np.linalg.inv(B))
print("coordinates of t-v in LLL basis: ")
print(m)

input point t: 
[ 4.2  7.1 -2.6]
Babai point v: 
[ 4.  7. -4.]
Euclidean norm of t-v: 
1.4177
coordinates of t-v in LLL basis: 
[-0.4  0.2  0.5]
